In [1]:
import sys
import os
from dataclasses import dataclass
from dotenv import load_dotenv
from tqdm import tqdm

sys.path.insert(0, "../src")

from extract_valid_studies import main
from models import ValidStudy, PrimaryOutcome

load_dotenv()

True

In [2]:
RAW_STUDIES_PATH = "../data/raw_studies"

In [26]:
studies = main(RAW_STUDIES_PATH)

100%|██████████| 34562/34562 [00:08<00:00, 4317.35it/s]


Loaded 5858 raw studies with results
2033 out of 5858 (34.70%) studies have p-values reported in primary outcomes analyses.


In [9]:
by_deck = {}
for study in studies:
    for deck in study.decks:
        if deck not in by_deck:
            by_deck[deck] = []
        by_deck[deck].append(study)

In [28]:
by_deck_counts = {deck: len(studies) for deck, studies in by_deck.items()}

for deck, studies in by_deck.items():
    n_succ = 0
    n_fail = 0
    n_any_succ = 0
    n_any_fail = 0
    perc_succ = []
    for study in studies:
        n_s_succ = 0
        n_s_fail = 0
        for outcome in study.primary_outcomes:
            success = outcome.check_success()
            if success is True:
                n_succ += 1
                n_s_succ += 1
            elif success is False:
                n_fail += 1
                n_s_fail += 1
        if n_s_succ > 0:
            n_any_succ += 1
        if n_s_fail > 0:
            n_any_fail += 1
        perc = n_s_succ / (n_s_succ + n_s_fail) if (n_s_succ + n_s_fail) > 0 else 0
        perc_succ.append(round(perc * 100))
    n_outcomes = n_succ + n_fail
    print(f"{deck}: {len(studies)} studies, {n_outcomes} outcomes, {n_succ/(n_succ+n_fail)*100 if n_fail > 0 else 0:.1f}%, {n_succ} successful outcomes, {n_fail} failed outcomes")
    print(f"  - Any successful studies: {n_any_succ} ({n_any_succ/len(studies)*100:.1f}%)")
    print(f"  - Any failed studies: {n_any_fail} ({n_any_fail/len(studies)*100:.1f}%)")
    print(f"  - Success percentages: avg {sum(perc_succ)/len(perc_succ):.1f}%")


Pain: 66 studies, 261 outcomes, 48.7%, 127 successful outcomes, 134 failed outcomes
  - Any successful studies: 36 (54.5%)
  - Any failed studies: 50 (75.8%)
  - Success percentages: avg 39.5%
Obesity: 117 studies, 340 outcomes, 43.5%, 148 successful outcomes, 192 failed outcomes
  - Any successful studies: 81 (69.2%)
  - Any failed studies: 64 (54.7%)
  - Success percentages: avg 56.9%
PTSD: 38 studies, 121 outcomes, 42.1%, 51 successful outcomes, 70 failed outcomes
  - Any successful studies: 23 (60.5%)
  - Any failed studies: 21 (55.3%)
  - Success percentages: avg 51.9%
Stress: 60 studies, 227 outcomes, 41.4%, 94 successful outcomes, 133 failed outcomes
  - Any successful studies: 41 (68.3%)
  - Any failed studies: 32 (53.3%)
  - Success percentages: avg 55.5%
Diabetes: 86 studies, 325 outcomes, 24.3%, 79 successful outcomes, 246 failed outcomes
  - Any successful studies: 50 (58.1%)
  - Any failed studies: 51 (59.3%)
  - Success percentages: avg 48.7%
Substance Use: 182 studies, 7

In [27]:
n = sum(len(s.primary_outcomes) for s in studies)
n_studies = len(studies)
n

7336